# Предобработка данных

Последовательная обработка

In [9]:
import pandas as pd
import re

df = pd.read_csv('raw.csv')

In [10]:
df.head(1)

,Unnamed: 0,Ссылка,Цена,Район,Адрес,Этаж,Высота потолка,Планировка,Площадь,Состояние квартиры,Кол-во комнат,Описание,Общая площадь,Жилая площадь,Площадь кухни,Этажность дома
0,23417266,/realty/prodazha-kvartir/prodam-kvartiru-pravo...,4890000.0,Правобережный,"Им. газеты ""Правда"", 62/2",2,NaN,NaN,"общая 62 м2, жилая 45 м2, кухни 8 м2",Хорошее,Трехкомнатная,💥 Квартира в очень хорошем состоянии. 👍\n\n✅ В...,62,45,8,5


In [11]:
df.tail(1)

,Unnamed: 0,Ссылка,Цена,Район,Адрес,Этаж,Высота потолка,Планировка,Площадь,Состояние квартиры,Кол-во комнат,Описание,Общая площадь,Жилая площадь,Площадь кухни,Этажность дома
503,14046477,/realty/prodazha-kvartir/prodam-kvartiru-pravo...,5000000.0,Правобережный,"Западное шоссе, 95",1,NaN,NaN,"общая 67,2 м2, жилая 45 м2, кухни 12 м2",NaN,Трехкомнатная,Срочно продам уникальную квартиру с дизайнерск...,"67,2",45,12,2


In [12]:
columns_to_drop = ['Unnamed: 0', 'Высота потолка', 'Планировка',
                   'Состояние квартиры', 'Площадь']
df.drop(columns=[col for col in columns_to_drop if col in df.columns],
        inplace=True)

df.tail(1)

,Ссылка,Цена,Район,Адрес,Этаж,Кол-во комнат,Описание,Общая площадь,Жилая площадь,Площадь кухни,Этажность дома
503,/realty/prodazha-kvartir/prodam-kvartiru-pravo...,5000000.0,Правобережный,"Западное шоссе, 95",1,Трехкомнатная,Срочно продам уникальную квартиру с дизайнерск...,"67,2",45,12,2


In [13]:
if 'Цена' in df.columns:
    df['Цена_руб'] = pd.to_numeric(
        df['Цена'].astype(str).str.replace(r'[^\d]', '', regex=True),
        errors='coerce'
    )
    df.drop(columns=['Цена'], inplace=True)
elif 'Цена руб.' in df.columns:
    df.rename(columns={'Цена руб.': 'Цена_руб'}, inplace=True)
    df['Цена_руб'] = pd.to_numeric(df['Цена_руб'], errors='coerce')

In [14]:
area_columns = ['Общая площадь', 'Жилая площадь', 'Площадь кухни']
for col in area_columns:
    if col in df.columns:
        df[f'{col}_м2'] = pd.to_numeric(
            df[col].astype(str).str.replace(',', '.'),
            errors='coerce'
        )
        df.drop(columns=[col], inplace=True)

df = df.rename(columns={
    'Общая площадь_м2': 'Площадь_общая_м2',
    'Жилая площадь_м2': 'Площадь_жилая_м2',
    'Площадь кухни_м2': 'Площадь_кухни_м2'
})

In [16]:
floor_columns = {'Этаж': 'Этаж_квартиры', 'Этажность дома': 'Этажность_дома'}
for old, new in floor_columns.items():
    if old in df.columns:
        df[new] = pd.to_numeric(df[old], errors='coerce').astype('Int64')
        df.drop(columns=[old], inplace=True)

In [17]:
room_mapping = {
    'Студия': 0, 'Однокомнатная': 1, 'Двухкомнатная': 2,
    'Трехкомнатная': 3, 'Четырехкомнатная': 4,
    'Пятикомнатная': 5, 'Многокомнатная': 5
}

if 'Кол-во комнат' in df.columns:
    df['Количество_комнат'] = df['Кол-во комнат'].map(room_mapping)
    df['Количество_комнат'] = pd.to_numeric(
        df['Количество_комнат'],
        errors='coerce'
    ).astype('Int64')
    df.drop(columns=['Кол-во комнат'], inplace=True)

In [ ]:
df.dropna(subset=['Цена_руб', 'Адрес', 'Количество_комнат',
                  'Площадь_общая_м2'], inplace=True)
print(f"After cleaning: {df.shape[0]} rows")

In [18]:
def clean_address(addr):
    if pd.isna(addr):
        return None

    addr = str(addr).strip().lower()
    addr = re.sub(r'г\.?\s*магнитогорск[,\s]*', '', addr)

    replacements = {
        r'\bпр-т\b': 'проспект',
        r'\bпр\.\b': 'проспект',
        r'\bул\.\b': 'улица',
        r'\bпер\.\b': 'переулок',
        r'\bд\.\s*': 'дом '
    }

    for pattern, replacement in replacements.items():
        addr = re.sub(pattern, replacement, addr)

    return re.sub(r'\s+', ' ', addr).strip()

df['Адрес'] = df['Адрес'].apply(clean_address)
df.dropna(subset=['Адрес'], inplace=True)

In [22]:
if 'Район' in df.columns:
    df['Район'] = (
        df['Район']
        .str.replace(r'\d+|\(.*?\)', '', regex=True)
        .str.strip()
        .replace({
            'Правобережный, 109': 'Правобережный',
            'Правобережный,': 'Правобережный',
            'Ленинский (Левый берег)': 'Ленинский',
            'Левобережный': 'Ленинский'
        })
    )

In [23]:
print("="*60)
print(f"FINAL DATASET: {df.shape[0]} rows × {df.shape[1]} columns")
print("="*60)

print("\nCOLUMNS & DATA TYPES:")
for col, dtype in df.dtypes.items():
    non_null = df[col].notna().sum()
    print(f"{col:25} {str(dtype):15} {non_null:>4} non-null")

print("\nNUMERICAL STATISTICS:")
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
if len(numeric_cols) > 0:
    display(df[numeric_cols].describe().round(2).T.style.background_gradient(cmap='Blues'))

FINAL DATASET: 499 rows × 11 columns

COLUMNS & DATA TYPES:
Ссылка                    object           499 non-null
Район                     object           185 non-null
Адрес                     object           499 non-null
Описание                  object           497 non-null
Цена_руб                  float64          498 non-null
Площадь_общая_м2          float64          499 non-null
Площадь_жилая_м2          float64          499 non-null
Площадь_кухни_м2          float64          499 non-null
Этаж_квартиры             Int64            499 non-null
Этажность_дома            Int64            499 non-null
Количество_комнат         Int64            491 non-null

NUMERICAL STATISTICS:


,count,mean,std,min,25%,50%,75%,max
Цена_руб,498.000000,46364810.040000,21216710.350000,2750000.000000,32380000.000000,41500000.000000,55390500.000000,152504520.000000
Площадь_общая_м2,499.000000,56.320000,25.720000,17.000000,42.000000,51.600000,65.000000,322.000000
Площадь_жилая_м2,499.000000,30.480000,18.270000,0.000000,18.000000,30.000000,40.000000,129.900000
Площадь_кухни_м2,499.000000,10.010000,5.120000,0.000000,6.000000,9.000000,12.000000,35.000000
Этаж_квартиры,499.000000,3.680000,2.550000,1.000000,2.000000,3.000000,5.000000,16.000000
Этажность_дома,499.000000,6.550000,3.850000,1.000000,4.000000,5.000000,9.000000,16.000000
Количество_комнат,491.000000,2.050000,0.850000,1.000000,1.000000,2.000000,3.000000,5.000000


In [24]:
print("\nCATEGORICAL DISTRIBUTIONS:")
if 'Район' in df.columns:
    display(df['Район'].value_counts().head(10).to_frame().style.bar(color='lightblue'))

print("\nSAMPLE ADDRESSES:")
print(df['Адрес'].head(5).tolist())


CATEGORICAL DISTRIBUTIONS:


,count
Район,
Орджоникидзевский,88
Правобережный,51
Ленинский,46



SAMPLE ADDRESSES:
['им. газеты "правда", 62/2', 'карла маркса, 112', 'октябрьская, 32/1', 'советская, 213/1', 'московская, 26/1']


In [25]:
output_path = 'cleaned_flats_magnitogorsk.csv'
df.to_csv(output_path, index=False)

## Итог

Итак, мы обработали наши данные и получили табличку с красивыми и чистыми значениями. С такой таблицей уже можно работать и не бояться, что что-либо исказит наши выводы.

Важно, что при предобработке, как и при анализе и создании моделей, нет правильного и неправильного решения. Каждое действие очень субъективно, способы обработки очень сильно разнятся от человека к человеку. Поэтому важно понимать причины и объяснять свои способы подготовки данных.

### Краткий итог предобработки

*   **Очистка структуры:** Удалены мусорные колонки (Unnamed, дублирующие признаки).
*   **Цена:** Извлечено числовое значение из строк.
*   **Стандартизация площадей:** Значения приведены к м², произведена замена запятых на точки.
*   **Преобразование этажей:** Данные переведены в целочисленный формат.
*   **Кодирование комнат:** Текстовые значения заменены на числа.
*   **Нормализация адресов:** Удалено упоминание города, стандартизированы сокращения.
*   **Очистка районов:** Удалены цифры и скобки, названия приведены к единому стандарту.
*   **Обработка пропусков:** Удалены записи с пустыми значениями в ключевых полях (цена, адрес, комнаты, площадь).
*   **Экспорт:** Результат сохранен в чистый CSV-файл.

**Итог:** Исходные сырые данные преобразованы в структурированный датасет для анализа.

In [27]:
df.head(2)

,Ссылка,Район,Адрес,Описание,Цена_руб,Площадь_общая_м2,Площадь_жилая_м2,Площадь_кухни_м2,Этаж_квартиры,Этажность_дома,Количество_комнат
0,/realty/prodazha-kvartir/prodam-kvartiru-pravo...,Правобережный,"им. газеты ""правда"", 62/2",💥 Квартира в очень хорошем состоянии. 👍\n\n✅ В...,48900000.0,62.0,45.0,8.0,2,5,3
1,/realty/prodazha-kvartir/prodam-kvartiru-23408...,NaN,"карла маркса, 112",Продается уютная двухкомнатная квартира на ком...,38000000.0,45.8,31.0,7.0,2,5,2


Теперь удобно искать топ по площади жилья, и тд

In [31]:
top_10_large = df.nlargest(10, 'Площадь_жилая_м2')

In [32]:
top_10_large.tail(10)

,Ссылка,Район,Адрес,Описание,Цена_руб,Площадь_общая_м2,Площадь_жилая_м2,Площадь_кухни_м2,Этаж_квартиры,Этажность_дома,Количество_комнат
219,/realty/prodazha-kvartir/prodam-kvartiru-22851...,NaN,"горнолыжная, 28",🏡 Уютная 3-комнатную квартира в посёлке Новоаб...,67500000.0,148.8,129.9,10.0,3,3,3
265,/realty/prodazha-kvartir/prodam-kvartiru-22233...,NaN,"калмыкова, 32а",КВАРТИРА БЛОКИРОВАННОЙ ЗАСТРОЙКИ площадью 156 ...,59900000.0,156.0,120.0,35.0,2,2,4
266,/realty/prodazha-kvartir/prodam-kvartiru-22181...,NaN,"калмыкова, 32а",Предлагаем вашему вниманию квартиры блокирован...,85000000.0,156.0,120.0,35.0,1,2,3
350,/realty/prodazha-kvartir/prodam-kvartiru-19874...,NaN,"галиуллина, 5/1",Единственная в своем роде кваpтиpa в тихом и з...,99000000.0,144.0,103.0,19.0,5,5,4
475,/realty/prodazha-kvartir/prodam-kvartiru-ordzh...,Орджоникидзевский,"тевосяна, 4а",Эксклюзивное предложение! <br />В продаже 4-ко...,146900000.0,178.7,102.0,10.9,3,6,4
376,/realty/prodazha-kvartir/prodam-kvartiru-18827...,NaN,ленина проспект,Выгодное вложение в свой отдых в самом красиво...,44000000.0,200.0,100.0,25.0,1,1,5
160,/realty/prodazha-kvartir/prodam-kvartiru-23028...,NaN,"калмыкова, 85",В продаже уникальная квартира приусадебного ти...,69000000.0,156.0,90.0,30.0,1,1,3
500,/realty/prodazha-kvartir/prodam-kvartiru-ordzh...,Орджоникидзевский,"карла маркса, 185",Предлагаю Вашему вниманию четырёхкомнатную ква...,80000000.0,105.0,83.0,10.0,4,9,4
483,/realty/prodazha-kvartir/prodam-kvartiru-ordzh...,Орджоникидзевский,"ленина проспект, 129",<p><b>ВНИМАНИЕ! </b><b>УНИКАЛЬНОЕ ПРЕДЛОЖЕНИЕ!...,95000000.0,105.5,80.0,16.0,7,16,3
299,/realty/prodazha-kvartir/prodam-kvartiru-21767...,NaN,"алмазная, 26",Продаётся уникальная 4-комнатная двухуровневая...,79748000.0,137.0,77.0,34.0,1,2,4
